# aims
* across group same role vs. same group across role vs. (null) across group across role
* 리콜해내는 이벤트는 둘 중 어떤 상황에서 더 유사할까?
* quantify1: event 간 similarity (content level)
* quantify2: across group role

1. 일단 각 그룹별 벌어졌던 이벤트를 쪼갬 (모든 그룹 sharedeb 준비)
2. Day1 각 이벤트의 content level vector를 구함 (only nv, USE 사용) -----quantify1
    1. USE 성능확인을 해봐야 함: 각 문장*을 넣고 pairwise vector similarity를 구하여 그것이 육안으로 확인한 내용적 유사성에 부합하는지
    2. *각 문장: raw, exclude stopwords, only nv 이렇게 세 단계로 테스트
3. 각 그룹의 각 이벤트에서 leader role(leader idx 1~3)을 정의하는 방법 구현(speaker와 NB_SBJ 등 구문분석 사용) -----quantify2
4. Day1에 대한 Day2의 리콜 태깅을 불러옴
5. 일단 각 그룹에서 전체 day1 이벤트들 대비 리콜된 이벤트들의 벡터 similarity에 특이점이 있는지 확인, across group 특이점이 있는지 확인 -----within group vector similarity가 월등히 높지 않을까 하는 걱정에서 비롯
6. (same group across role) 리콜된 day2의 이벤트들의 day1 벡터들의 similarity 계산 -----pairwise로 전부?
7. (across group same role) 리콜된 day2의 이벤트들의 day1 벡터들의 similarity 계산 -----pairwise로 전부; 위와 동일하게
8. (across group across role) 리콜된 day2의 이벤트들의 day1 벡터들의 similarity 계산 -----pairwise로 전부; null로 사용

# USE

In [28]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
from scipy.spatial import distance

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
    return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [90]:
DATADIR = '/home/jiunchoi/OFD/OFD_BHV_clean'
group = 5
sub = 1
groupsub = '0'+str(group)+'0'+str(sub)

names = []
with open(f"{DATADIR}/subnames", "r") as f:
    names.extend(f.read().split('\n'))

subname = names[group-1].split()
subname = subname[1:]

In [91]:
data_transcript = pd.read_excel(f'{DATADIR}/group-0{str(group)}/group-0{str(group)} sharedeb.xlsx')
data_script = data_transcript['sentence']
sharedeb = np.array(data_transcript['sharedeb'])
sentence_embedding = np.array(embed(data_script))

event_semantic_vectors = np.zeros((int(sharedeb[-1]),sentence_embedding.shape[-1]))
for i in range (1,int(sharedeb[-1])+1):
    event_semantic_vectors[i-1] = np.mean(sentence_embedding[sharedeb==i,:],axis=0)

In [92]:
np.save(f'{DATADIR}/group-0{str(group)}/group-0{str(group)} event_semantic_vectors.npy',event_semantic_vectors)
event_semantic_vectors.shape

(193, 512)

In [182]:
group=2
keys = [f'sub-0{group}01',f'sub-0{group}02',f'sub-0{group}03']

data_transcript = pd.read_excel(f'{DATADIR}/group-0{str(group)}/group-0{str(group)} sharedeb.xlsx')
sharedeb = np.array(data_transcript['sharedeb'])
data_recs = np.array(data_transcript[keys])
notrecalled_evidx=[]

for i in range(len(sharedeb)):
    count=0
    for j in range(3):
        if type(data_recs[i,j])!=str and np.isnan(data_recs[i,j])==True:
            count+=1
    if count==3:
        if sharedeb[i] not in notrecalled_evidx:
            notrecalled_evidx.append(sharedeb[i])

recalled_evidx=[]

for i in range(len(sharedeb)):
    count=0
    for j in range(3):
        if type(data_recs[i,j])==str or np.isnan(data_recs[i,j])==False:
            count+=1
    if count==3:
        if sharedeb[i] not in recalled_evidx:
            recalled_evidx.append(sharedeb[i])

[14,
 15,
 16,
 17,
 31,
 32,
 33,
 34,
 40,
 44,
 45,
 52,
 53,
 54,
 55,
 59,
 60,
 61,
 63,
 64,
 65,
 83,
 84,
 88,
 90,
 96,
 97,
 98,
 111,
 118,
 119]